In [2]:
# imports
from snowexsql.db import get_db
from snowexsql.data import SiteData, ImageData, LayerData, PointData
from snowexsql.conversions import query_to_geopandas
import datetime

In [3]:
# load the database
db_name = 'snow:hackweek@52.32.183.144/snowex'
engine, session = get_db(db_name)

## Snow Pits
### Example 1: 
grab all pits that overlap with lidar day(s)

1). Have a known date in mind that you wish to select, here's how:

2). Unsure of the date, but know which instrument you'd like to overlap with, here's how:

In [8]:
qry = session.query(ImageData.surveyors)

In [11]:
# Form on the Images table that returns Raster collection dates
qry = session.query(ImageData.date)

# Filter by ASO to get the Lidar flights 
qry = qry.filter(ImageData.surveyors == 'ASO Inc.')

# Grab the unique dates
qry = qry.distinct()

# Execute the query 
dates = qry.all() 

# Clean up the dates 
dates = [d[0] for d in dates]

# Pick a day 
dt = dates[1]


# Form a date range to query on either side of our chosen day 
date_range = [dt + i * datetime.timedelta(days=1) for i in [-1, 0, 1]]


# Find all the pits done on these days 
qry = session.query(SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date.in_(date_range))
results = qry.all()
print(results)


[('1C5', datetime.date(2020, 2, 12)), ('FL1B', datetime.date(2020, 2, 12)), ('2C12', datetime.date(2020, 2, 12)), ('5N24', datetime.date(2020, 2, 12)), ('6S26', datetime.date(2020, 2, 12)), ('2C13', datetime.date(2020, 2, 12)), ('5N32', datetime.date(2020, 2, 12))]


In [ ]:
# Great for debugging especially when trying different queries
session.rollback()

# Find all the data that was collected on 2-1-2020
dt = datetime.date(2020, 2, 1)

# Grab all point data instruments from our date
point_instruments = session.query(PointData.instrument).filter(PointData.date == dt).distinct().all()
print(point_instruments)

# Grab all Layer data instruments from our date
layer_instruments = session.query(LayerData.instrument, LayerData.type).filter(LayerData.date == dt).distinct().all()
print(layer_instruments)

# Grab all Image data instruments from our date
image_instruments = session.query(ImageData.instrument).filter(ImageData.date == dt).distinct().all()
print(image_instruments)


### Example 2: 
grab all pits based on flight line pattern

1). only want to analyize pits from the 'N' flight line?

2). what about the 'N' flight line for a 1 week period?

In [ ]:
# Grab density
import matplotlib.pyplot as plt 

session.rollback()

q = session.query(LayerData).filter(LayerData.type == 'density').filter(LayerData.site_id.contains('N')).limit(100)
df = query_to_geopandas(q, engine)
# Convert density to float
df['value'] = df['value'].astype(float)

df 

df.plot()
plt.show()


# Calculate SWE
# swe_lambda = lambda row: row['value'] * (row['depth'] - row['bottom_depth']) / 100
# df['swe'] = df.apply(swe_lambda, axis=1)

In [ ]:
df

### Example 3: 
grab all pits based on the depth and vegetation matrix

1). interested in isolating shallow pits? here's how: 

2). what about pits in dense canopy cover? 

In [ ]:
## Depths